# Using Knowledge Graph as a tool

[Knowledge Graph](https://dev.writer.com/api-guides/knowledge-graph) is Writer's graph-based retrieval-augmented generation (RAG). With the Knowledge Graph tool, you can access a Knowledge Graph in a chat with a Palmyra LLM.

This cookbook shows how to use the Knowledge Graph tool to answer specific questions during a chat with an AI assistant.

## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)
- A Knowledge Graph. If you don't already have one, you can create one by following the [Knowledge Graph Cookbook](https://github.com/writer/cookbooks/blob/main/knowledge_graph/knowledge_graph.ipynb) or the [Knowledge Graph tutorial](https://dev.writer.com/api-guides/knowledge-graph).
    - You'll need the ID of the Knowledge Graph you want to use as a tool. You can find the ID via the [API](https://dev.writer.com/api-guides/api-reference/kg-api/list-graphs) or via the [Writer AI Studio](https://app.writer.com/aistudio/knowledge-graph) UI. The ID is the last part of the URL for the Knowledge Graph: `https://app.writer.com/aistudio/organization/<organization_id>/knowledge-graph?graphId=<graph_id>`

## Setup

Install the Writer SDK:

In [ ]:
%pip install -qU writer-sdk

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()  # The Writer client automatically uses the WRITER_API_KEY environment variable to authenticate.

## Usage

The Knowledge Graph tool follows this format:

```json
{
    "type": "graph",
    "function": {
        "description": "Description of the Knowledge Graph",
        "graph_ids": ["graph_id"],
        "subqueries": True
    }
}
```

The `description` field should be a short description of what the Knowledge Graph contains.

The `subqueries` field indicates whether the response should include the subqueries that were sent to the Knowledge Graph.

The following code snippet asks you to input the ID of the Knowledge Graph you want to use as a tool and the description of the Knowledge Graph, and it constructs the `tools` array with this information.

In [ ]:
graph_id = input("Enter the ID of the Knowledge Graph you want to use as tool: ")
graph_description = input("Enter a description of the Knowledge Graph you want to use as tool: ")

tools = [
    {
        "type": "graph",
        "function": {
            "description": graph_description,
            "graph_ids": [graph_id],
            "subqueries": True
        }
    }
]

The next code snippet asks you to input a question to answer from your Knowledge Graph, and then asks that question to the LLM. It passes the Knowledge Graph tool to the LLM.

In [ ]:
user_question = input("Enter a question to answer from your Knowledge Graph: ")

response = client.chat.chat(
    model="palmyra-x-004",
    messages=[{"role": "user", "content": user_question}],
    tools=tools,
    tool_choice="auto"
)

print(f"Answer: {response.choices[0].message.content}")

If the Knowledge Graph was used in the response, you can see that in the `response.choices[0].message.graph_data` field.

In [ ]:
print(f"Graph data: {response.choices[0].message.graph_data.to_json()}")


If there were any subqueries to the Knowledge Graph to answer the question, you can see them in the `response.choices[0].message.graph_data.subqueries` field.

In [ ]:
print(f"Subqueries: {response.choices[0].message.graph_data.subqueries}")

## Streaming

You can also stream the chat response, which will stream the response as it is generated. The following code snippet shows how to stream the response, collect the text as it is generated, and print it out.

In [ ]:
response_stream = client.chat.chat(
    model="palmyra-x-004",
    messages=[{"role": "user", "content": user_question}],
    tools=tools,
    tool_choice="auto",
    stream=True
)

output_text = ""

for chunk in response_stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
        output_text += chunk.choices[0].delta.content

print(f"Final output: {output_text}")